<a href="https://colab.research.google.com/github/BODDUSRIPAVAN/FMML_Project_and_Labs/blob/main/Copy_of_FMML_CLF1_Lab3_2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **FOUNDATIONS OF MODERN MACHINE LEARNING, IIIT Hyderabad**
### MODULE: CLASSIFICATION-1
### LAB-3 : Using KNN for Text Classification
#### Module Coordinator: Tanvi Kamble


---

## **Section 1: Understanding NLP tools** 

In this lab we will be using KNN on a real world NLP application i.e. is text classification. But first look at some NLP techniques for text classification and tools that we use when we want to use python for NLP.

## Section 1.2: Data Cleaning and Preprocessing step

Raw text must be processed and converted into a form so that it is suitable to use with various machine-learning algorithms.  
In case of text, there are lots of things that need to be taken into account.  


1.   Removing numbers from the text
2.   Handling capitalization and punctuation.
3.   Stemming and Lemmatizing text.  

And most importantly, one can't just use words or images directly in algorithms; they need to be converted into vectors- a form that algorithms can understand.



### **NLTK** 
NLTK (or Natural Language Tool Kit) is a commonly used library for processing text. We will use this tool in this lab. Lets first install it. 


In [100]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')
import warnings as warn
warn.filterwarnings("ignore")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [101]:
import re
import numpy
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from bs4 import BeautifulSoup

def cleanText(text, lemmatize, stemmer):
    """Method for cleaning text from train and test data. Removes numbers, punctuation, and capitalization. Stems or lemmatizes text."""

    if isinstance(text, float):
        text = str(text)
    if isinstance(text, numpy.int64):
        text = str(text)
    try:
        text = text.decode()
    except AttributeError:
        pass

    soup = BeautifulSoup(text, "lxml")
    text = soup.get_text()
    text = re.sub(r"[^A-Za-z]", " ", text)
    text = text.lower()


    if lemmatize:
        wordnet_lemmatizer = WordNetLemmatizer()

        def get_tag(tag):
            if tag.startswith('J'):
                return wordnet.ADJ
            elif tag.startswith('V'):
                return wordnet.VERB
            elif tag.startswith('N'):
                return wordnet.NOUN
            elif tag.startswith('R'):
                return wordnet.ADV
            else:
                return ''

        text_result = []
        tokens = word_tokenize(text)  # Generate list of tokens
        tagged = pos_tag(tokens)
        for t in tagged:
            try:
                text_result.append(wordnet_lemmatizer.lemmatize(t[0], get_tag(t[1][:2])))
            except:
                text_result.append(wordnet_lemmatizer.lemmatize(t[0]))
        return text_result

    if stemmer:
        text_result = []
        tokens = word_tokenize(text)
        snowball_stemmer = SnowballStemmer('english')
        for t in tokens:
            text_result.append(snowball_stemmer.stem(t))
        return text_result

In [102]:
sample_text = "Troubling"
sample_text_result = cleanText(sample_text, lemmatize=False, stemmer=True)
sample_text_result = " ".join(str(x) for x in sample_text_result)
print(sample_text)
print(sample_text_result)
sample_text_result = cleanText(sample_text, lemmatize=True, stemmer=False)
sample_text_result = " ".join(str(x) for x in sample_text_result)
print(sample_text_result)

Troubling
troubl
trouble


## Section 1.2: BAG OF WORDS 

A bag-of-words model, or BoW for short, is a way of extracting features from text for use in modeling, such as with machine learning algorithms.

The approach is very simple and flexible, and can be used in many ways for extracting features from documents.

A bag-of-words is a representation of text that describes the occurrence of words within a document. 
It is called a “bag” of words, because any information about the order or structure of words in the document is discarded. The model is only concerned with whether known words occur in the document, not where in the document.

In [103]:
# Functions to convert document(s) to a list of words, with the option of removing stopwords. Returns document-term matrix.

def createBagOfWords(train, test, remove_stopwords, lemmatize, stemmer):
    if remove_stopwords:
        vectorizer = CountVectorizer(analyzer='word', input='content', stop_words=stopwords.words('english'))
    else:
        vectorizer = CountVectorizer(analyzer='word', input='content')

    clean_train = []
    for paragraph in train:
        paragraph_result = cleanText(paragraph, lemmatize, stemmer)
        paragraph = " ".join(str(x) for x in paragraph_result)
        clean_train.append(paragraph)

    clean_test = []
    for paragraph in test:
        paragraph_result = cleanText(paragraph, lemmatize, stemmer)
        paragraph = " ".join(str(x) for x in paragraph_result)
        clean_test.append(paragraph)

    bag_of_words_train = vectorizer.fit_transform(clean_train).toarray()
    bag_of_words_test = vectorizer.transform(clean_test).toarray()
    return bag_of_words_train, bag_of_words_test


## Section 1.3: TF-IDF 
TF-IDF technique is used to find meaning of sentences consisting of words and cancels out the incapabilities of Bag of Words technique which is good for text classification or for helping a machine read words in numbers.

The number of times a term occurs in a document is called its Term frequency (TF).

 Document frequency is the number of documents in which the word is present.  Inverse DF (IDF) is the inverse of the document frequency which measures the informativeness of term *t*. 




In [105]:
def createTFIDF(train, test, remove_stopwords, lemmatize, stemmer):
    if remove_stopwords:
        vectorizer = TfidfVectorizer(analyzer='word', input='content', stop_words=stopwords.words('english'))
    else:
        vectorizer =  TfidfVectorizer(analyzer='word', input='content')

    clean_train = []
    for paragraph in train:
        paragraph_result = cleanText(paragraph, lemmatize, stemmer)
        paragraph = " ".join(str(x) for x in paragraph_result)
        clean_train.append(paragraph)

    clean_test = []
    for paragraph in test:
        paragraph_result = cleanText(paragraph, lemmatize, stemmer)
        paragraph = " ".join(str(x) for x in paragraph_result)
        clean_test.append(paragraph)

    tfidf_train = vectorizer.fit_transform(clean_train).toarray()
    tfidf_test = vectorizer.transform(clean_test).toarray()
    return tfidf_train, tfidf_test

# **Section 2: UNDERSTANDING THE DATA : A REVIEWS DATASET**

Sentiment analysis is the interpretation and classification of emotions (such as positive, negative and neutral) within text data using text analysis techniques.  
Given below is a dataset consisting of reviews along with sentiment class (positive or negative).

In [106]:
# Upload the Reviews CSV file that has been shared with you.
# Run this cell, click on the 'Choose files' button and upload the file.
from google.colab import files
uploaded = files.upload()

Saving reviews.csv to reviews (1).csv


In [108]:
import pandas as pd
df = pd.read_csv('reviews.csv')

In [109]:
df = df.dropna()

In [110]:
df.to_csv('reviews.csv', index=False)

In [111]:
df["sentiment"].unique()

array(['0', '1', ' it just lacked imagination.',
       ' nothing about where he spend 2 years between his childhood and mature age.'],
      dtype=object)

# **Section 3: KNN MODEL**

Given below are two KNN models; in the first case we are using Bag-of-Words and in the second case we are using TF-IDF.
Note the different metrics and parameters used in each.

In [112]:
from sklearn import metrics, neighbors
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict

## TASK - 1: Tweak the models below and see results with different parameters and distance metrics.

def bow_knn():
    """Method for determining nearest neighbors using bag-of-words and K-Nearest Neighbor algorithm"""

    training_data = pd.read_csv('reviews.csv')
    X_train, X_test, y_train, y_test = train_test_split(training_data["sentence"], training_data["sentiment"], test_size=0.2, random_state=5)
    X_train, X_test = createBagOfWords(X_train, X_test, remove_stopwords=True, lemmatize=True, stemmer=False)
    # print(X_train)
    knn = neighbors.KNeighborsClassifier(n_neighbors=5, weights='uniform', algorithm='auto', leaf_size=30, p=2, metric='euclidean', metric_params=None, n_jobs=1)

    knn.fit(X_train, y_train)
    predicted = knn.predict(X_test)
    acc = metrics.accuracy_score(y_test, predicted)
    print('KNN with BOW accuracy = ' + str(acc * 100) + '%')

    scores = cross_val_score(knn, X_train, y_train, cv=3)
    print("Cross Validation Accuracy: %0.2f" % (scores.mean()))
    print(scores)
    print('\n')
    return predicted, y_test


def tfidf_knn():
    """Method for determining nearest neighbors using tf-idf and K-Nearest Neighbor algorithm"""

    training_data = pd.read_csv('reviews.csv')
    X_train, X_test, y_train, y_test = train_test_split(training_data["sentence"], training_data["sentiment"],
                                                        test_size=0.2, random_state=5)
    X_train, X_test = createTFIDF(X_train, X_test, remove_stopwords=True, lemmatize=True, stemmer=False)
    # print(X_train)
    knn = neighbors.KNeighborsClassifier(n_neighbors=5, weights='distance', algorithm='brute', leaf_size=30, p=2,
                                         metric='cosine', metric_params=None, n_jobs=1)

    knn.fit(X_train, y_train)
    predicted = knn.predict(X_test)
    acc = metrics.accuracy_score(y_test, predicted)
    print('KNN with TFIDF accuracy = ' + str(acc * 100) + '%')

    scores = cross_val_score(knn, X_train, y_train, cv=3)
    print("Cross Validation Accuracy: %0.2f" % (scores.mean()))
    print(scores)
    return predicted, y_test

Note: Cross-validation will be discussed in detail in the upcoming lab session.

In [114]:
## KNN accuracy after using BoW
predicted, y_test = bow_knn()

KNN with BOW accuracy = 62.30366492146597%
Cross Validation Accuracy: 0.62
[0.60784314 0.58431373 0.66141732]




In [115]:
## KNN accuracy after using TFIDF
predicted, y_test = tfidf_knn()

KNN with TFIDF accuracy = 70.15706806282722%
Cross Validation Accuracy: 0.73
[0.7254902  0.74117647 0.72834646]


# TASK-1 : Tweak the models below and see results with different parameters and distance metrics.

In [116]:
def bow_knn(a,b,c):
    training_data = pd.read_csv('reviews.csv')
    X_train, X_test, y_train, y_test = train_test_split(training_data["sentence"], training_data["sentiment"], test_size=0.2, random_state=5)
    X_train, X_test = createBagOfWords(X_train, X_test, remove_stopwords=True, lemmatize=True, stemmer=False)
    # print(X_train)
    knn = neighbors.KNeighborsClassifier(n_neighbors=a, weights=b, algorithm='auto', leaf_size=30, p=2, metric=c, metric_params=None, n_jobs=1)

    knn.fit(X_train, y_train)
    predicted = knn.predict(X_test)
    acc = metrics.accuracy_score(y_test, predicted)
    print('KNN with BOW accuracy = ' + str(acc * 100) + '%')

    scores = cross_val_score(knn, X_train, y_train, cv=3)
    print("Cross Validation Accuracy: %0.2f" % (scores.mean()))
    print(scores)
    print('\n')

In [117]:
bow_knn(10,"uniform","hamming")

KNN with BOW accuracy = 59.16230366492147%
Cross Validation Accuracy: 0.62
[0.63921569 0.58039216 0.64173228]




In [118]:
bow_knn(10,"distance","hamming")

KNN with BOW accuracy = 64.92146596858639%
Cross Validation Accuracy: 0.64
[0.66666667 0.59607843 0.65748031]




In [119]:
bow_knn(8,"uniform","correlation")

KNN with BOW accuracy = 70.68062827225131%
Cross Validation Accuracy: 0.70
[0.67843137 0.66666667 0.74015748]




In [120]:
bow_knn(8,"distance","correlation")

KNN with BOW accuracy = 68.06282722513089%
Cross Validation Accuracy: 0.72
[0.71764706 0.69411765 0.7519685 ]




In [121]:
bow_knn(9,"distance","jaccard")

KNN with BOW accuracy = 71.72774869109948%
Cross Validation Accuracy: 0.72
[0.70588235 0.71764706 0.73228346]




In [122]:
def tfidf_knn(a,b,c):
    """Method for determining nearest neighbors using tf-idf and K-Nearest Neighbor algorithm"""

    training_data = pd.read_csv('reviews.csv')
    X_train, X_test, y_train, y_test = train_test_split(training_data["sentence"], training_data["sentiment"],
                                                        test_size=0.2, random_state=5)
    X_train, X_test = createTFIDF(X_train, X_test, remove_stopwords=True, lemmatize=True, stemmer=False)
    # print(X_train)
    knn = neighbors.KNeighborsClassifier(n_neighbors=a, weights=b, algorithm='brute', leaf_size=30, p=2,
                                         metric=c, metric_params=None, n_jobs=1)

    knn.fit(X_train, y_train)
    predicted = knn.predict(X_test)
    acc = metrics.accuracy_score(y_test, predicted)
    print('KNN with TFIDF accuracy = ' + str(acc * 100) + '%')

    scores = cross_val_score(knn, X_train, y_train, cv=3)
    print("Cross Validation Accuracy: %0.2f" % (scores.mean()))
    print(scores)

In [123]:
tfidf_knn(1,"uniform","euclidean")

KNN with TFIDF accuracy = 61.25654450261781%
Cross Validation Accuracy: 0.58
[0.52156863 0.67843137 0.53543307]


In [124]:
tfidf_knn(10,"uniform","jaccard")

KNN with TFIDF accuracy = 67.5392670157068%
Cross Validation Accuracy: 0.70
[0.69019608 0.70588235 0.69685039]


In [125]:
tfidf_knn(15,"distance","hamming")

KNN with TFIDF accuracy = 54.45026178010471%
Cross Validation Accuracy: 0.54
[0.54509804 0.51764706 0.57086614]


In [126]:
tfidf_knn(10,"uniform","yule")

KNN with TFIDF accuracy = 70.15706806282722%
Cross Validation Accuracy: 0.68
[0.68627451 0.69803922 0.65748031]


In [128]:
tfidf_knn(20,"distance","cosine")

KNN with TFIDF accuracy = 76.96335078534031%
Cross Validation Accuracy: 0.77
[0.75294118 0.75686275 0.78740157]


# Section 4: SPAM TEXT DATASET
Now let's use what we've learnt to classify texts as spam or not spam.

In [129]:
# Upload the spam text data CSV file that has been shared with you. You can also download the file from https://www.kaggle.com/datasets/uciml/sms-spam-collection-dataset
# Run this cell, click on the 'Choose files' button and upload the file.
from google.colab import files
uploaded = files.upload()

Saving spam.csv to spam (1).csv


In [130]:
import pandas as pd
df = pd.read_csv('spam.csv', error_bad_lines=False)
df

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ã¼ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [131]:
df['Category'] = df['Category'].map({'ham': 0, 'spam': 1})

In [132]:
df.head(5)

,Category,Message
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [133]:
len(df)

5572

In [134]:
from sklearn import metrics, neighbors
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict

## TASK - 2: Tweak the models below and see results with different parameters and distance metrics.

def bow_knn():
    """Method for determining nearest neighbors using bag-of-words and K-Nearest Neighbor algorithm"""

    training_data = pd.read_csv('spam.csv')
    training_data['Category'] = training_data['Category'].map({'ham': 0, 'spam': 1})
    X_train, X_test, y_train, y_test = train_test_split(training_data["Message"], training_data["Category"], test_size=0.2, random_state=5)
    X_train, X_test = createBagOfWords(X_train, X_test, remove_stopwords=True, lemmatize=True, stemmer=False)
    knn = neighbors.KNeighborsClassifier(n_neighbors=5, weights='uniform', algorithm='auto', leaf_size=30, p=2, metric='euclidean', metric_params=None, n_jobs=1)

    knn.fit(X_train, y_train)
    predicted = knn.predict(X_test)
    acc = metrics.accuracy_score(y_test, predicted)
    print('KNN with BOW accuracy = ' + str(acc * 100) + '%')

    scores = cross_val_score(knn, X_train, y_train, cv=3)
    print("Cross Validation Accuracy: %0.2f" % (scores.mean()))
    print(scores)
    print('\n')
    return predicted, y_test


def tfidf_knn():
    """Method for determining nearest neighbors using tf-idf and K-Nearest Neighbor algorithm"""

    training_data = pd.read_csv('spam.csv')
    training_data['Category'] = training_data['Category'].map({'ham': 0, 'spam': 1})
    X_train, X_test, y_train, y_test = train_test_split(training_data["Message"], training_data["Category"], test_size=0.2, random_state=5)
    X_train, X_test = createTFIDF(X_train, X_test, remove_stopwords=True, lemmatize=True, stemmer=False)
    knn = neighbors.KNeighborsClassifier(n_neighbors=5, weights='distance', algorithm='brute', leaf_size=30, p=2, metric='cosine', metric_params=None, n_jobs=1)

    knn.fit(X_train, y_train)
    predicted = knn.predict(X_test)
    acc = metrics.accuracy_score(y_test, predicted)
    print('KNN with TFIDF accuracy = ' + str(acc * 100) + '%')

    scores = cross_val_score(knn, X_train, y_train, cv=3)
    print("Cross Validation Accuracy: %0.2f" % (scores.mean()))
    print(scores)
    return predicted, y_test

In [135]:
# This cell may take some time to run
predicted, y_test = bow_knn()

KNN with BOW accuracy = 92.19730941704036%
Cross Validation Accuracy: 0.91
[0.90713324 0.90040377 0.91245791]




In [136]:
# This cell may take some time to run
predicted, y_test = tfidf_knn()

KNN with TFIDF accuracy = 98.56502242152466%
Cross Validation Accuracy: 0.97
[0.96837147 0.96769852 0.96363636]


# TASK - 2: Tweak the models below and see results with different 
# parameters and distance metrics.


In [137]:
def bow_knn(a,b,c):
    """Method for determining nearest neighbors using bag-of-words and K-Nearest Neighbor algorithm"""

    training_data = pd.read_csv('spam.csv')
    training_data['Category'] = training_data['Category'].map({'ham': 0, 'spam': 1})
    X_train, X_test, y_train, y_test = train_test_split(training_data["Message"], training_data["Category"], test_size=0.2, random_state=5)
    X_train, X_test = createBagOfWords(X_train, X_test, remove_stopwords=True, lemmatize=True, stemmer=False)
    knn = neighbors.KNeighborsClassifier(n_neighbors=a, weights=b, algorithm='auto', leaf_size=30, p=2, metric=c, metric_params=None, n_jobs=1)

    knn.fit(X_train, y_train)
    predicted = knn.predict(X_test)
    acc = metrics.accuracy_score(y_test, predicted)
    print('KNN with BOW accuracy = ' + str(acc * 100) + '%')

    scores = cross_val_score(knn, X_train, y_train, cv=3)
    print("Cross Validation Accuracy: %0.2f" % (scores.mean()))
    print(scores)
    print('\n')
    return predicted, y_test


def tfidf_knn(a,b,c):
    """Method for determining nearest neighbors using tf-idf and K-Nearest Neighbor algorithm"""

    training_data = pd.read_csv('spam.csv')
    training_data['Category'] = training_data['Category'].map({'ham': 0, 'spam': 1})
    X_train, X_test, y_train, y_test = train_test_split(training_data["Message"], training_data["Category"], test_size=0.2, random_state=5)
    X_train, X_test = createTFIDF(X_train, X_test, remove_stopwords=True, lemmatize=True, stemmer=False)
    knn = neighbors.KNeighborsClassifier(n_neighbors=a, weights=b, algorithm='brute', leaf_size=30, p=2, metric=c, metric_params=None, n_jobs=1)

    knn.fit(X_train, y_train)
    predicted = knn.predict(X_test)
    acc = metrics.accuracy_score(y_test, predicted)
    print('KNN with TFIDF accuracy = ' + str(acc * 100) + '%')

    scores = cross_val_score(knn, X_train, y_train, cv=3)
    print("Cross Validation Accuracy: %0.2f" % (scores.mean()))
    print(scores)
    return predicted, y_test

In [138]:
bow_knn(1,"uniform","hamming")

KNN with BOW accuracy = 94.97757847533632%
Cross Validation Accuracy: 0.94
[0.9461642  0.93539704 0.94276094]




(array([1, 0, 1, ..., 0, 1, 0]), 2095    1
 5343    0
 564     1
 3849    0
 3317    0
        ..
 3473    0
 884     0
 5006    0
 3397    1
 1309    0
 Name: Category, Length: 1115, dtype: int64)

In [139]:
bow_knn(10,"distance","jaccard")

KNN with BOW accuracy = 98.56502242152466%
Cross Validation Accuracy: 0.98
[0.97779273 0.98183042 0.97373737]




(array([1, 0, 1, ..., 0, 1, 0]), 2095    1
 5343    0
 564     1
 3849    0
 3317    0
        ..
 3473    0
 884     0
 5006    0
 3397    1
 1309    0
 Name: Category, Length: 1115, dtype: int64)

In [140]:
bow_knn(3,"uniform","cosine")

KNN with BOW accuracy = 98.29596412556054%
Cross Validation Accuracy: 0.96
[0.95625841 0.96567968 0.96632997]




(array([1, 0, 1, ..., 0, 1, 0]), 2095    1
 5343    0
 564     1
 3849    0
 3317    0
        ..
 3473    0
 884     0
 5006    0
 3397    1
 1309    0
 Name: Category, Length: 1115, dtype: int64)

In [141]:
tfidf_knn(2,"uniform","hamming")

KNN with TFIDF accuracy = 88.5201793721973%
Cross Validation Accuracy: 0.87
[0.87617766 0.87146703 0.87474747]


(array([1, 0, 0, ..., 0, 0, 0]), 2095    1
 5343    0
 564     1
 3849    0
 3317    0
        ..
 3473    0
 884     0
 5006    0
 3397    1
 1309    0
 Name: Category, Length: 1115, dtype: int64)

In [142]:
tfidf_knn(10,"uniform","jaccard")

KNN with TFIDF accuracy = 97.847533632287%
Cross Validation Accuracy: 0.97
[0.96702557 0.97510094 0.96835017]


(array([1, 0, 1, ..., 0, 1, 0]), 2095    1
 5343    0
 564     1
 3849    0
 3317    0
        ..
 3473    0
 884     0
 5006    0
 3397    1
 1309    0
 Name: Category, Length: 1115, dtype: int64)

In [143]:
tfidf_knn(20,"distance","euclidean")

KNN with TFIDF accuracy = 91.65919282511211%
Cross Validation Accuracy: 0.94
[0.93001346 0.93808883 0.94478114]


(array([1, 0, 1, ..., 0, 1, 0]), 2095    1
 5343    0
 564     1
 3849    0
 3317    0
        ..
 3473    0
 884     0
 5006    0
 3397    1
 1309    0
 Name: Category, Length: 1115, dtype: int64)

### Questions to Think About and Answer
1. Why does the TF-IDF approach generally result in a better accuracy than Bag-of-Words ?
2. Can you think of techniques that are better than both BoW and TF-IDF ?
3. Read about Stemming and Lemmatization from the resources given below. Think about the pros/cons of each.

# 1. Why does the TF-IDF approach generally result in a better accuracy than Bag-of-Words ?
## * TF-IDF stands for "Term Frequency-Inverse Document Frequency"
### It is a numerical statistic that is intended to reflect how important a word is to a document in a corpus(collection of texts/ documents)
### High weight tfidf is given to those terms which occur within specific documents and rarely used across all documents.
### Low weight tfidf is given to those terms which occur commonly across all documents. 
### So, it identifies and captures what are the words belongs to spam and not spam.

### * BoW stands for Bag of Words
### Bag of Words creates a set of vectors containing the count of word occurrences in the document (reviews)
### Bag of Words vectors are easy to interpret. 
## However, TF-IDF usually performs better than BoW because, it contains information on the more important words and the less important ones as well whereas BoW only count frequency of words in the document.


#2.Can you think of techniques that are better than both BoW and TF-IDF ?
## * Disadvantages:-
### -> Both the BoW and TF-IDF techniques are unable to capture "Semantic Similarity", i.e., how similar two worlds/ sentences in their contextual meaning.
### -> They are unable to capture collocations, distributional similarity, associations.
### Ex:- Dog-Puppy, Lion-Cub, Deer-Elk
## * Yes, there are techniques that are better than both BoW and TF-IDF. They are:-
### 1) PLSA = Probabilistic Latent Semantic Analysis (used in topic modelling)
### 2) LDA = Latent Dirichlent Allocation (used in topic modelling)
### 3) Word2Vec,
### 4) Continuous Bag of Words (CBOW),
### 5) Skipgram, etc.,

# 3. Read about Stemming and Lemmatization from the resources given below. Think about the pros/cons of each.
# *** Stemming
## * Pros:-
### 1) Stemming is the process of reducing a word to its stem that affixes to suffixes and prefixes or to the roots of words known as "lemmas" or it can be defined as the process of finding the root word of any given word by removing morphological variances.
### 2) Simple and easy to understand
### 3) Root word can be extracted
### 4) Strictly dictionary based or rule based stemmers (e.g. Porter stemmer) are very fast.
### 5) Increases performance of the model
## * Cons:-
### 1) Most of the popular implementations for stemming are still rule based. Not all the words can be stemmed to their right root word.
### 2) Stemming can't relate words which have different forms based on grammatical constructs, like is, am, be, all represent the same root verb, be. Stemming can't prune them to the common form. The word better should be resolved to good, but stemmers would fail to do that.
### 3) With stemming, there is lot of ambiguity which may cause several false positives. Axes is both a plural form of axe and axis. By chopping of the "s", you won't be able to relate x axis in a plane with x-y axes.
### 4) Further, stemming is harder for several languages like Samskrutham ,Hebrew and Arabic.
### 5) Sometimes it may generate unexpected and meaningless root words.
# *** Lemmatization
## * Pros:-
### 1) Lemmatization is the process of converting the words of a sentence to its dictionary form or it can be defined as the process of grouping together different forms of same word where we want to have the words that come out to be actually meaningful. For example, given the words amusement, amusing, and amused, the lemma for each and all would be amuse.
### 2) The main advantage of lemmatization is that it takes into consideration the context of the word to determine which is the intended meaning the user is looking for.
### 3) Meaningful root words are extracted.
### 4) This process allows to decrease noise and speed up the user's task.
### 5) It is more accurate.
## * Cons:-
### 1) Because lemmatization involves deriving the meaning of a word from something like a dictionary, it's very time consuming.
### 2) So most lemmatization algorithms are slower compared to their stemming counterparts.
### 3) Semantic Similarity cannot be measured.

### Useful Resources for further reading
1. Stemming and Lemmatization: https://nlp.stanford.edu/IR-book/html/htmledition/stemming-and-lemmatization-1.html
2. TF-IDF and BoW : https://www.analyticsvidhya.com/blog/2020/02/quick-introduction-bag-of-words-bow-tf-idf/
3. TF-IDF: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html
